<center><font color='steelblue'> <font size = "5,5">CORPORACIÓN UNIVERSITARIA MINUTO DE DIOS </font></center><br>
<center><font color='steelblue'> <font size = "5">FACULTAD DE INGENIERÍA</font></center><br>
<center><font color='steelblue'> <font size = "4">PROGRAMA INGENIERÍA DE SISTEMAS</font></center><br>
<center><font color='steelblue'> <font size = "3">CURSO BASES DE DATOS MASIVAS</font></center><br>
<center><font color="yellow" size = "4" face = "small fonts">Proyecto Modular - Mojo</font></center>

<center><font color="olive" size = "4" face = "small fonts">DATOS DE LOS PARTICIPANTES DEL GRUPO</font></center>

<font color="yellow" size = "4" face = "small fonts">NRC: 80132</font></center><br>
<font color="yellow" size = "4" face = "small fonts">Nombres: Laura Tatiana Bernal Yanquen, Daniel Yesid Casallas Páez</font></center><br>
<font color="yellow" size = "4" face = "small fonts">ID:857437</font></center><br>

<font color="yellow" size = "4" face = "small fonts">El taller retoma algunas de las instrucciones utilizadas a través del curso de Bases de Datos Masivas, tener en cuenta seguir los pasos requeridos.</font></center><br>

<font color="yellow" size = "4" face = "small fonts">Al Cargar los archivos dispuestos en la carpeta Poryecto final BDM se debe utilizar sentencias que no fijen el path, este debe se dinámico en caso de que se creen nuevos paquetes</font></center><br>

<font color="red" size = "4" face = "small fonts">El ejercicio consta de seguir el cuaderno y  realizar las tareas solicitadas, Utilizar y citar la documentación propia de cada una de las herramientas utilizadas para realizar el tratamiento a la data.</font></center><br>

<font color="yellow" size = "4" face = "small fonts">1. Configuración e importe de las librerias.</font></center><br>

In [104]:
# Esta extencion permite que jupyter detecte cambios externos y recargue sin tener que reiniciar VSC
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
# Importar la librearia os para poder interactuar el sistema donde se esta ejecutando los archivos
import os
import requests
import re
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from collections import defaultdict
import json

<font color="yellow" size = "4" face = "small fonts">2. Configure y levante los contenedores necesarios para la actividad, recuerde que la arquitectura respeta a 3 contenedores con una distribucion de DBMongo en cada una de ellas, un contenedor que tiene configurado Mojolicious con todas sus dependencias junto con una base de datos Relacional la cual sera escogida a su gusto.</font></center><br>

In [ ]:
# Moverse a la carpeta data donde estan los archivos para lanzar el docker
os.chdir("../data")
# Crear y ejecutar el docker-compose.yml
os.system("docker-compose up --build")

<font color="yellow" size = "4" face = "small fonts">3. Una vez tenga el contenedor arriba cargue los datos .Json, uno por cada contenedor DBMongo.</font></center><br>

<font color="yellow" size = "4" face = "small fonts">Tenga en cuenta que debe tener la replica de los formatos `JSon` en cada contenedor</font></center><br>

In [106]:
# Con estos comandos se deben cargar los datos .json dentro de las mongoDB sin necesidad de una funcion.

# Comando para importar personas.json
os.system("docker cp ../data/personas.json core_container:/app/data/personas.json")

# Comando para importar articulos.json
os.system("docker cp ../data/articulos.json core_container:/app/data/articulos.json")

# Comando para importar ventas.json
os.system("docker cp ../data/ventas.json core_container:/app/data/ventas.json") 

0

<font color="yellow" size = "4" face = "small fonts">Se debe colocar el puerto expuesto del contenedor `Mojo`, para llamar la función y cargar los datos</font></center><br>

In [107]:
# Cargar data a las Mongo a con la funcion
requests.get("http://localhost:8080/load_data")

<Response [200]>

<font color="yellow" size = "4" face = "small fonts">4. Muestre los datos que fueron almacenados en cada una de las distribuciones de BDMongo.</font></center><br>

In [ ]:
# Verificar conteo de documentos en la colección personas
os.system("")

# Verificar conteo de documentos en la colección articulos
os.system("")

# Verificar conteo de documentos en la colección ventas
os.system("")

In [ ]:
os.chdir("../data")
from etl_paralelo import ejecutar_etl_paralelo

ejecutar_etl_paralelo()

🚀 ETL PARALELO - PROCESAMIENTO DE ALTO RENDIMIENTO
💻 CPUs disponibles: 8

PASO 1/3: ARTÍCULOS
🚀 Procesando 10000 artículos con 8 workers...
🚀 Procesando 10000 artículos con 8 workers...


<font color="yellow" size = "4" face = "small fonts">Consultar a tráves del puerto expuesto la data que se encuentran en las colecciones personas, articulos y ventas de DBMongo</font></center><br>

In [ ]:
# Ver personas en MongoDB
personas = requests.get("http://localhost:8080/mongo/personas")
print(json.dumps(personas.json(), ensure_ascii=False, indent=2))

In [ ]:
# Ver articulos en MongoDB
articulos = requests.get("http://localhost:8080/mongo/articulos")
print(json.dumps(articulos.json(), ensure_ascii=False, indent=2))

In [ ]:
# Ver ventas en MongoDB
ventas = requests.get("http://localhost:8080/mongo/ventas")
print(json.dumps(ventas.json(), ensure_ascii=False, indent=2))

<font color="yellow" size = "4" face = "small fonts">5. Verifique la estructura y tipo de los datos almacenados en las distribuciones DBMongo, genere una ETL para almacenar cada una de las distribuciones en la base de datos Relacional que configuro con anterioridad, recuerde que cada distribucion debe ir en una tabla relacional respetando la integridad referencial.</font></center><br>

In [ ]:
# Funciones en Python para obtener datos de MongoDB para articulos
def limpiar_valor_numerico(valor):
    
    if valor is None:
        return None
    
    # Convertir a string y limpiar espacios
    valor_str = str(valor).strip()
    
    # Eliminar caracteres no numéricos (excepto el signo negativo al inicio)
    valor_limpio = re.sub(r'[^\d-]', '', valor_str)
    
    try:
        return int(valor_limpio)
    except (ValueError, TypeError):
        return None
    


# Función para verificar y limpiar idArticulo y cantidadArticulo donde debe ser un dato entero, tener en cuenta elimiar el id que proporciona 
# DBMongo a cada uno de las colecciones para cargarla a la Bases de Datos Relacional en SQLlite que importo.
def filtrar_y_limpiar_articulos(datos):
    # Definir los campos que necesitan ser numéricos
    campos_a_limpiar = ['idArticulo', 'cantidadArticulo']
    datos_limpios = []
    
    # Primera fase: Limpiar datos
    for documento in datos:
        # Eliminar el _id de MongoDB
        if '_id' in documento:
            del documento['_id']
        
        campos_validos = True
        
        # Validar y limpiar campos numéricos
        for campo in campos_a_limpiar:
            valor_limpio = limpiar_valor_numerico(documento.get(campo))
            
            if valor_limpio is None:
                campos_validos = False
                break
            
            documento[campo] = valor_limpio
        
        # Limpiar precioArticulo (puede ser float)
        if 'precioArticulo' in documento:
            try:
                precio_str = str(documento['precioArticulo']).strip()
                documento['precioArticulo'] = float(re.sub(r'[^\d.-]', '', precio_str))
            except (ValueError, TypeError):
                campos_validos = False
        
        if campos_validos:
            datos_limpios.append(documento)
    
    # Segunda fase: Consolidar artículos por nombre
    articulos_por_nombre = {}
    
    for articulo in datos_limpios:
        nombre = articulo.get('nombreArticulo', '').strip()
        
        if nombre not in articulos_por_nombre:
            articulos_por_nombre[nombre] = {
                'nombreArticulo': nombre,
                'cantidadArticulo': articulo['cantidadArticulo'],
                'suma_precios': articulo.get('precioArticulo', 0),
                'contador': 1,
                'ids_anteriores': [articulo['idArticulo']]
            }
        else:
            articulos_por_nombre[nombre]['cantidadArticulo'] += articulo['cantidadArticulo']
            articulos_por_nombre[nombre]['suma_precios'] += articulo.get('precioArticulo', 0)
            articulos_por_nombre[nombre]['contador'] += 1
            articulos_por_nombre[nombre]['ids_anteriores'].append(articulo['idArticulo'])
    
    # Tercera fase: Crear nuevos IDs y mapeo
    articulos_consolidados = []
    mapeo_ids = {}
    nuevo_id = 1
    
    for articulo in articulos_por_nombre.values():
        articulo['precioArticulo'] = round(articulo['suma_precios'] / articulo['contador'], 2)
        articulo['idArticulo'] = nuevo_id
        
        # Guardar mapeo de nuevo ID a IDs anteriores
        mapeo_ids[nuevo_id] = articulo['ids_anteriores']
        
        # Eliminar campos auxiliares
        del articulo['suma_precios']
        del articulo['contador']
        del articulo['ids_anteriores']
        
        articulos_consolidados.append(articulo)
        nuevo_id += 1
    
    return articulos_consolidados, mapeo_ids


# Uso de la función
response = requests.get("http://localhost:8080/mongo/articulos")
articulos = response.json()
articulos_limpios, mapeo_ids = filtrar_y_limpiar_articulos(articulos)
print(json.dumps(articulos_limpios, ensure_ascii=False, indent=2))
print(f"\nMapeo de IDs (nuevo -> anteriores): {mapeo_ids}")

# Insertar en SQLite
for articulo in articulos_limpios:
    try:
        response = requests.post("http://localhost:8080/sqlite/articulos", json=articulo)
        if response.status_code != 200:
            print(f"Error al insertar artículo {articulo.get('idArticulo')}: {response.json()}")
    except Exception as e:
        print(f"Excepción al insertar artículo: {e}")

In [96]:
# Obtener datos de MongoDB para personas, escriba un ETL donde haga la validación de numeroDocumento y telefono, asegurando que solo contengan valores 
# numéricos, tener en cuenta elimiar el id que proporciona DBMongo a cada uno de las colecciones para cargarla a la Bases de Datos Relacional en SQLlite que importo.

def filtrar_limpiar_personas(personas, ventas, mapeo_ids):
    """
    ETL robusto sin _norm_num_str:
    - Normaliza documento y teléfono.
    - Elimina _id de Mongo.
    - Solo conserva personas con ventas enlazables.
    - Resuelve duplicados con sufijo incremental estable.
    - Mapea idArticulo con mapeo_ids (normalizado).
    - Descarta ventas sin comprador enlazable o con campos inválidos.
    Retorna (personas_limpias, ventas_finales).
    """

    # ---- 0) Mapeo de artículos: invertir + normalizar a int ----
    mapeo_inverso = {}
    for nuevo_id, ids_anteriores in (mapeo_ids or {}).items():
        try:
            nuevo_int = int(str(nuevo_id))
        except Exception:
            continue
        for old in ids_anteriores or []:
            try:
                old_int = int(str(old))
                mapeo_inverso[old_int] = nuevo_int
            except Exception:
                continue

    # ---- 1) Preparar personas con id temporal y clave normalizada ----
    personas_con_id_temp = []
    id_temporal = 1

    # doc_key (str del documento normalizado) -> lista[id_temp]
    dockey_a_ids_temp = defaultdict(list)

    for p in personas:
        pc = dict(p)
        pc.pop('_id', None)

        num_doc = limpiar_valor_numerico(pc.get('numeroDocumento'))
        if num_doc is None:
            # No podremos emparejar esta persona
            continue

        doc_key = str(num_doc)  # <- reemplaza _norm_num_str

        pc['_id_temp'] = id_temporal
        pc['_doc_key'] = doc_key
        dockey_a_ids_temp[doc_key].append(id_temporal)
        personas_con_id_temp.append(pc)
        id_temporal += 1

    # ---- 2) Preasignar id_temp a ventas según comprador normalizado ----
    ventas_con_id_temp = []
    for v in ventas:
        vc = dict(v)
        vc.pop('_id', None)

        comp_num = limpiar_valor_numerico(vc.get('idComprador'))
        if comp_num is not None:
            comp_key = str(comp_num)  # <- reemplaza _norm_num_str
            if dockey_a_ids_temp.get(comp_key):
                vc['_id_temp_comprador'] = dockey_a_ids_temp[comp_key][0]  # determinista (primero)

        ventas_con_id_temp.append(vc)

    # ---- 3) Limpiar ventas y filtrar las enlazables ----
    ventas_limpias = []
    compradores_necesarios = set()

    for v in ventas_con_id_temp:
        # Campos numéricos obligatorios
        id_venta    = limpiar_valor_numerico(v.get('idVenta'))
        id_articulo = limpiar_valor_numerico(v.get('idArticulo'))
        cantidad    = limpiar_valor_numerico(v.get('cantidadProductos'))

        if id_venta is None or id_articulo is None or cantidad is None:
            continue

        # Mapear artículo si corresponde
        if id_articulo in mapeo_inverso:
            id_articulo = mapeo_inverso[id_articulo]

        # precioTotal robusto (permite símbolos)
        raw_precio = str(v.get('precioTotal', 0))
        raw_precio = re.sub(r'[^\d.-]', '', raw_precio) or '0'
        try:
            precio_total = round(float(raw_precio), 2)
        except Exception:
            continue

        # Necesitamos un id_temp de comprador válido para garantizar FK
        id_temp_comp = v.get('_id_temp_comprador')
        if not id_temp_comp:
            # Venta sin persona enlazable → descartar (evita FK rota)
            continue

        compradores_necesarios.add(id_temp_comp)

        # Cargar con placeholders; luego reemplazamos idComprador con doc final
        v_clean = {
            'idVenta': id_venta,
            'idComprador': None,  # se asigna en el paso 5
            'idArticulo': id_articulo,
            'cantidadProductos': cantidad,
            'precioTotal': precio_total,
            '_id_temp_comprador': id_temp_comp
        }
        ventas_limpias.append(v_clean)

    # ---- 4) Construir personas_limpias SOLO para compradores necesarios ----
    personas_limpias = []
    docs_usados = set()
    contador_dups = defaultdict(int)  # num_doc_limpio -> contador
    id_temp_a_doc_final = {}

    for p in personas_con_id_temp:
        if p['_id_temp'] not in compradores_necesarios:
            continue

        num_doc = limpiar_valor_numerico(p.get('numeroDocumento'))
        if num_doc is None:
            continue

        # Resolver duplicados de forma determinista con sufijo incremental
        if num_doc in docs_usados:
            contador_dups[num_doc] += 1
            num_doc_final = int(f"{num_doc}{contador_dups[num_doc]}")
        else:
            contador_dups[num_doc] = 0
            num_doc_final = num_doc

        docs_usados.add(num_doc_final)
        id_temp_a_doc_final[p['_id_temp']] = num_doc_final

        # Teléfono solo dígitos o None
        tel = p.get('telefono')
        if tel is not None:
            tel_digits = re.sub(r'[^\d]', '', str(tel).strip())
            p['telefono'] = tel_digits if tel_digits else None
        else:
            p['telefono'] = None

        # Asignar documento final y limpiar temporales
        p['numeroDocumento'] = num_doc_final
        p.pop('_id_temp', None)
        p.pop('_doc_key', None)

        personas_limpias.append(p)

    # ---- 5) Actualizar ventas con documento final del comprador ----
    ventas_finales = []
    for v in ventas_limpias:
        id_temp = v.pop('_id_temp_comprador', None)
        doc_final = id_temp_a_doc_final.get(id_temp)
        if not doc_final:
            # No debería ocurrir (ya filtramos), pero por seguridad
            continue
        v['idComprador'] = doc_final
        ventas_finales.append(v)

    return personas_limpias, ventas_finales



personas = requests.get("http://localhost:8080/mongo/personas").json()
ventas = requests.get("http://localhost:8080/mongo/ventas").json()

personas_limpias, ventas_limpias = filtrar_limpiar_personas(personas, ventas, mapeo_ids)

# Insertar en SQLite
for persona in personas_limpias:
    try:
        response = requests.post("http://localhost:8080/sqlite/personas", json=persona)
        if response.status_code != 200:
            print(f"Error al insertar persona {persona.get('numeroDocumento')}: {response.json()}")
    except Exception as e:
        print(f"Excepción al insertar persona: {e}")

for venta in ventas_limpias:
    try:
        response = requests.post("http://localhost:8080/sqlite/ventas", json=venta)
        if response.status_code != 200:
            print(f"Error al insertar venta {venta.get('idVenta')}: {response.json()}")
    except Exception as e:
        print(f"Excepción al insertar venta: {e}")

In [ ]:
# Obtener datos de MongoDB para ventas, escriba un ETL donde haga la validación de idComprador, idArticulo y cantidadProductos asegurando que solo contengan valores 
# numéricos, tener en cuenta elimiar el id que proporciona DBMongo a cada uno de las colecciones para cargarla a la Bases de Datos Relacional en SQLlite que importo.


# Obtener datos de MongoDB para ventas
response = requests.get("http://localhost:8080/mongo/ventas")
ventas = response.json()
ventas_limpias = filtrar_y_limpiar(ventas, 'idVenta', 'idComprador', 'idArticulo', 'cantidadProductos')
print(json.dumps(ventas_limpias, ensure_ascii=False, indent=2))

<font color="yellow" size = "4" face = "small fonts">6. Muestre la data almacenada en la base de datos Relacional que esta usando, tabla por tabla.</font></center><br>

In [ ]:
# Mostrar data almacenada en la tabla personas
# Conectar a la base de datos SQLite
conexion = sqlite3.connect('../data/data/almacen.sqlite')

# Crear una consulta SQL y cargar los resultados en un DataFrame
consulta = "SELECT * FROM personas;"

df = pd.read_sql_query(consulta, conexion)

# Mostrar los datos
print(df)

# Cerrar la conexión
conexion.close()


In [ ]:
# Mostrar data almacenada en la tabla articulos
# Conectar a la base de datos SQLite
conexion = sqlite3.connect('../data/data/almacen.sqlite')

# Crear una consulta SQL y cargar los resultados en un DataFrame
consulta = "SELECT * FROM articulos"
df = pd.read_sql_query(consulta, conexion)

# Mostrar los datos
print(df)

# Cerrar la conexión
conexion.close()

In [ ]:
# Mostrar data almacenada en la tabla ventas
# Conectar a la base de datos SQLite
conexion = sqlite3.connect('../data/data/almacen.sqlite')

# Crear una consulta SQL y cargar los resultados en un DataFrame
consulta = "SELECT * FROM ventas;"
df = pd.read_sql_query(consulta, conexion)

# Mostrar los datos
print(df)

# Cerrar la conexión
conexion.close()

<font color="yellow" size = "4" face = "small fonts">7. Grafique los 5 articulos mas vendidos.</font></center><br>

In [ ]:
# Conectar a la base de datos
conexion = sqlite3.connect('../data/data/almacen.sqlite') 

  # Ejecutar la consulta y cargar los datos en un DataFrame
consulta = """SELECT
    a.idArticulo,
    a.nombreArticulo,
    SUM(v.cantidadProductos) AS unidades_vendidas,
    SUM(v.precioTotal)      AS ingresos_totales
    FROM ventas v
    JOIN articulos a ON a.idArticulo = v.idArticulo
    GROUP BY a.idArticulo, a.nombreArticulo
    ORDER BY unidades_vendidas DESC, ingresos_totales DESC, a.nombreArticulo ASC
    LIMIT 5;"""


df = pd.read_sql_query(consulta, conexion)

# Cerrar la conexión
conexion.close()

# Graficar los datos
plt.figure(figsize=(12, 6))
plt.bar(df['nombreArticulo'], df['unidades_vendidas'], color='skyblue')
plt.title('Top 5 Artículos Más Vendidos')
plt.xlabel('Artículo')
plt.ylabel('Unidades Vendidas')
plt.xticks(rotation=45)
plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))

plt.tight_layout()
plt.show()



<font color="yellow" size = "4" face = "small fonts">8. Grafique los 5 compradores que han realizado más compras.</font></center><br>

In [ ]:
# Conectar a la base de datos
conexion = sqlite3.connect('../data/data/almacen.sqlite') 

# Ejecutar la consulta y cargar los datos en un DataFrame
consulta = """
SELECT
  p.numeroDocumento,
  p.nombres,
  p.primerApellido,
  p.segundoApellido,
  COUNT(*)                           AS compras_realizadas,
  SUM(v.cantidadProductos)           AS unidades_compradas,
  SUM(v.precioTotal)                 AS total_gastado
FROM ventas v
JOIN personas p ON p.numeroDocumento = v.idComprador
GROUP BY p.numeroDocumento, p.nombres, p.primerApellido, p.segundoApellido
ORDER BY unidades_compradas DESC, total_gastado DESC, p.primerApellido ASC, p.nombres ASC
LIMIT 5;
"""
df = pd.read_sql_query(consulta, conexion)


# Asegura tipos y arma el nombre completo de forma segura
# Crear nombre completo del cliente
df['cliente'] = (df['nombres'] + ' ' + df['primerApellido'] + ' ' + df['segundoApellido']).str.strip()

# Graficar
plt.figure(figsize=(12, 6))
plt.bar(df['cliente'], df['total_gastado'], color='lightcoral')
plt.title('Top 5 Compradores que Más Han Comprado')
plt.xlabel('Cliente')
plt.ylabel('Total Gastado ($)')
plt.xticks(rotation=45, ha='right')
plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
plt.tight_layout()
plt.show()

# Cerrar la conexión
conexion.close()


<font color="yellow" size = "4" face = "small fonts">9. Grafique la distribución de precios de los 5 primeros artículos .</font></center><br>

In [ ]:
# Conectar a la base de datos
conexion = sqlite3.connect('../data/data/almacen.sqlite') 

# Ejecutar la consulta y cargar los datos en un DataFrame
consulta = """
SELECT
  idArticulo,
  nombreArticulo,
  precioArticulo
FROM articulos
ORDER BY idArticulo ASC
LIMIT 5;
"""
df = pd.read_sql_query(consulta, conexion)

# Graficar
plt.figure(figsize=(10, 6))
plt.bar(df['nombreArticulo'], df['precioArticulo'], color='lightgreen')
plt.title('Distribución de Precios de los 5 Primeros Artículos')
plt.xlabel('Artículo')
plt.ylabel('Precio ($)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Cerrar la conexión
conexion.close()


<font color="yellow" size = "4" face = "small fonts">10. Grafique los 5 artículos que menos se han vendido.</font></center><br>

In [ ]:
# Conectar a la base de datos
conexion = sqlite3.connect('../data/data/almacen.sqlite') 

# Ejecutar la consulta y cargar los datos en un DataFrame
consulta = """
SELECT
    a.idArticulo,
    a.nombreArticulo,
    COALESCE(SUM(v.cantidadProductos), 0) AS unidades_vendidas,
    COALESCE(SUM(v.precioTotal), 0) AS ingresos_totales
FROM articulos a
LEFT JOIN ventas v ON a.idArticulo = v.idArticulo
GROUP BY a.idArticulo, a.nombreArticulo
ORDER BY unidades_vendidas ASC, ingresos_totales ASC, a.nombreArticulo ASC
LIMIT 5;
"""
df = pd.read_sql_query(consulta, conexion)

# Cerrar la conexión
conexion.close()

# Graficar los datos
plt.figure(figsize=(12, 6))
plt.bar(df['nombreArticulo'], df['unidades_vendidas'], color='salmon')
plt.title('Top 5 Artículos Menos Vendidos')
plt.xlabel('Artículo')
plt.ylabel('Unidades Vendidas')
plt.xticks(rotation=45, ha='right')
plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
plt.tight_layout()
plt.show()